In [1]:
#enviroment
# !pip install openai
# !pip install spacy
# !python -m spacy download zh_core_web_sm


import openai
from openai import OpenAI
import os
import spacy

OPENAI_API_KEY = 'sk-proj-o9TONJi0MW2tSiDMhRkxT3BlbkFJkUr03XQ5IfUaxamV0e3k'

You should consider upgrading via the 'C:\Users\mekae\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\mekae\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')


You should consider upgrading via the 'c:\Users\mekae\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Entity Recognition, Glossary Databse(can be replaced with knowledge graph) Part.

In [21]:
nlp = spacy.load("zh_core_web_sm")

glossary_db = {
    '黑光纤': 'dark fiber',
    '光纤': 'optical fiber',
}

def identify_entities(source_text):
    doc = nlp(source_text)
    entities = []
    translations = {}
    for ent in doc.ents:
        term = ent.text.lower()
        if term in glossary_db:
            translations[ent.text] = glossary_db[term]
    return translations

text = "光纤照上去变成黑光纤了"
translations = identify_entities(text)

print("Identified entities:", translations)

Identified entities: {'黑光纤': 'dark fiber'}


Prompt generation based on idea of RAG.

In [22]:
# prompt with entity translation
def prompt_generator(text, source_language, target_language):
  terms = identify_entities(text)
  terms_translations = '. '.join([f"'{k}': '{v}'" for k, v in terms.items()])
  translations = f"Based on the translation of following term: {terms_translations}."
  prompt = f"Translate the following text from {source_language} into {target_language}: {text}\n"
  if terms == {}:
    return prompt
  prompt = translations + prompt

  return prompt

prompt = prompt_generator('光纤照上去变成黑光纤了', 'zh', 'en')
print(prompt)

Based on the translation of following term: '黑光纤': 'dark fiber'.Translate the following text from zh into en: 光纤照上去变成黑光纤了



Machine Translation code, can be replaced by other models.

In [23]:
def translate_text(prompt):

    client = OpenAI(api_key= OPENAI_API_KEY,)
    print(prompt)
    response = client.chat.completions.create(
      messages=[{
            "role": "user",
            "content": prompt,
          }],
      model="gpt-3.5-turbo",)
    translation = response.choices[0].message.content.strip().split("\n")[0]
    return translation

In [25]:
#test
prompt = prompt_generator('光纤照上去变成黑光纤了', 'zh', 'en')
translation = translate_text(prompt)
print(translation)

Based on the translation of following term: '黑光纤': 'dark fiber'.Translate the following text from zh into en: 光纤照上去变成黑光纤了



AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************0e3k. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

Quality Estimation Part

In [ ]:
def quality_estimator(original_text, translated_text):
  client = OpenAI(api_key= OPENAI_API_KEY,)
  prompt = f"Evaluate the quality estimation of the following source and translation sentence pairs by following a step-by-step process: \
    Step 1: Estimate the perplexity of the translated sentence.\
    Step 2: Determine the token-level similarity between the source and translatedsentences.\
    Step 3: Combine the results and classify the translation quality into one of the following categories:'No meaning preserved', 'Some meaning preserved, but not understandable', 'Some meaning preserved and understandable', 'Most meaningpreserved, minor issues',or 'Perfect translation'.\
    Source:{original_text}.Translation:{translated_text}"
  print(prompt)
  response = client.chat.completions.create(
    messages=[{
          "role": "user",
          "content": prompt,
        }],
    model="gpt-3.5-turbo",
    )
  result = response.choices[0].message.content
  return result

In [ ]:
def quality_classifier(evaluation):
  start_index = evaluation.find("'")
  end_index = evaluation.find("'", start_index + 1)
  category = evaluation[start_index+1:end_index]
  return category

In [ ]:
original_text = "光纤照上去变成黑光纤了"
translated_text = "The fiber optic cable shines and turns into dark fiber."
evaluation = quality_estimator(original_text, translated_text)
print(evaluation)

Evaluate the quality estimation of the following source and translation sentence pairs by following a step-by-step process:     Step 1: Estimate the perplexity of the translated sentence.    Step 2: Determine the token-level similarity between the source and translatedsentences.    Step 3: Combine the results and classify the translation quality into one of the following categories:'No meaning preserved', 'Some meaning preserved, but not understandable', 'Some meaning preserved and understandable', 'Most meaningpreserved, minor issues',or 'Perfect translation'.    Source:光纤照上去变成黑光纤了.Translation:The fiber optic cable shines and turns into dark fiber.
Step 1: Estimate the perplexity of the translated sentence.
The translated sentence is grammatically correct and coherent, so the perplexity is relatively low.

Step 2: Determine the token-level similarity between the source and translated sentences.
Source tokens: 光纤 照 上去 变成 黑 光纤 了
Translation tokens: The fiber optic cable shines and turns

In [ ]:
category = quality_classifier(evaluation)
print(category)

Some meaning preserved and understandable


In [ ]:
def refine_prompt(original_text, translated_text, evaluation):
  prompt = f"Based on the quality estimation of the following source and translation sentence pairs:{original_text}.Translation:{translated_text}.Category:{evaluation}, provide a refined translation:"
  return prompt


In [ ]:
prompt = refine_prompt(original_text, translated_text, evaluation)
new_translation = translate_text(prompt)
print(new_translation)

Based on the quality estimation of the following source and translation sentence pairs:光纤照上去变成黑光纤了.Translation:The fiber optic cable shines and turns into dark fiber..Category:Step 1: Estimate the perplexity of the translated sentence.
The translated sentence is grammatically correct and coherent, so the perplexity is relatively low.

Step 2: Determine the token-level similarity between the source and translated sentences.
Source tokens: 光纤 照 上去 变成 黑 光纤 了
Translation tokens: The fiber optic cable shines and turns into dark fiber
Token-level similarity: The tokens do not match exactly, but the overall meaning is preserved. 

Step 3: Combine the results and classify the translation quality
Based on the low perplexity and the preservation of the overall meaning, the translation quality can be classified as 'Some meaning preserved and understandable'. The translation conveys the general idea of the source sentence, but there are some discrepancies in word choice and accuracy., provide a re

User iterface action: edit context(providing few shot for style guide) and rephrase(generate alternative translation)

In [ ]:
# Few-shot context changing via user input
def edit_context(context, original_text, source_language, target_language, few_shot_examples):

  edited_context = f'''Translate "{original_text}" from the {source_language} language, into the {target_language} language
                      using the following context style: "{context}".
                      Here are some examples of the style being used: {few_shot_examples}
                      '''
  return translate_text(source_language, target_language, edited_context)


# Chinese to Chinese Example
print("CHINESE TO ENGLISH EXAMPLE")

# Poetic Example
examples = '''"Light speeds away, in a blink it will fly."
              "Stars reach from afar, with light from eons bold."
           '''

print(edit_context("A poetic two line rhyming style", "光纤照上去变成黑光纤了", "zh", "en", examples), "\n\n\n")

# Confused and Inquisitive Example
examples = '''"How does it work so well when it's so unclear?"
              "Why did it begin so suddenly, what triggered it?"
            '''

print(edit_context("A confused and inquisitive style", "光纤照上去变成黑光纤了", "zh", "en", examples), "\n\n\n")



# English to Chinese Example
print("ENGLISH TO CHINESE EXAMPLE")

# Poetic Example
examples = '''"光逝如飞，瞬息即逝。"
              "星光遥来，古光犹豪。"
           '''

print(edit_context("A poetic two line rhyming style", "Lose track of time I'm bugging", "en", "zh", examples), "\n\n\n")

# Confused and Inquisitive Example
examples = '''"当它如此不清楚时，它如何运作得这么好？"
              "为什么它开始得这么突然，是什么触发了它？"
            '''

print(edit_context("A confused and inquisitive style", "Lose track of time I'm bugging", "en", "zh", examples))


CHINESE TO ENGLISH EXAMPLE
Translate "光纤照上去变成黑光纤了" from the zh language, into the en language
                      using the following context style: "A poetic two line rhyming style".
                      Here are some examples of the style being used: "Light speeds away, in a blink it will fly."
              "Stars reach from afar, with light from eons bold."
           
                      
"Fiber glass shining bright, turns into darkness in the light." 



Translate "光纤照上去变成黑光纤了" from the zh language, into the en language
                      using the following context style: "A confused and inquisitive style".
                      Here are some examples of the style being used: "How does it work so well when it's so unclear?"
              "Why did it begin so suddenly, what triggered it?"
            
                      
How did the fiber optic become black when it was lit up? 



ENGLISH TO CHINESE EXAMPLE
Translate "Lose track of time I'm bugging" from the en languag

In [ ]:

# Feeding the model (OpenAI API) the original translation meta descriptors and prompting for a new/rephrased/alternate translation
def rephrase(old_translation, original_source_text, source_language, target_language):

  # Get entites so they don't have to be passed, simplifying the function call. Can be changed to passing if more efficient
  terms = identify_entities(original_source_text)
  entities_joined = '. '.join([f"'{k}': '{v}'" for k, v in terms.items()])

  # Construct the prompt for rephrased/alternate translation
  prompt = f'''Please provide an alternative translation to and rephrase: "{old_translation}"
          based on the term {entities_joined}.
          The original text was {original_source_text}.
          The source language is {source_language} and the target language is {target_language}.'''

  # Make the call to the model
  return translate_text(prompt)

# Get alternative translation for demo text
rephrase("The fiber optic turned into dark fiber when the light shone on it.", "光纤照上去变成黑光纤了", "zh", "en")


# Probably want to use the Translation memory/storage for avoiding repeat translations and get unique results each time

Please provide an alternative translation to and rephrase: "The fiber optic turned into dark fiber when the light shone on it." 
          based on the term '黑光纤': 'dark fiber'.
          The original text was 光纤照上去变成黑光纤了.
          The source language is zh and the target language is en.


"The optical fiber transformed into dark fiber under the light's illumination."

Interactive code for testing(pipeline)

In [ ]:
# Get user input
text = input("Enter the text to translate: ")
source_language = input("Enter the source language: ")
target_language = input("Enter the target language: ")
prompt = prompt_generator(source_language, target_language, prompt)
translation = translate_text(prompt)
evaluation = quality_estimator(text, translation)
category = quality_classifier(evaluation)
if category == 'Most meaningpreserved, minor issues' or category == 'Perfect translation':
  print('Accepted')
else:
  translation = refine_prompt(text, translation, evaluation)
print(translation)

KeyboardInterrupt: Interrupted by user